In [2]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd


In [3]:
# get events from n days ago
iso = 562 #Look in ACLED documentation ISO code for number this is for Niger
limit = 400

api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}'.format(iso)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

https://api.acleddata.com/acled/read?terms=accept&iso=562 <class 'str'>


In [4]:
response = requests.get(api_url)
data = response.json()
data.keys()
data['count'] 

500

### From the documentation we know this is the max return --- How can we get all the results?

In [5]:
# Let's mkae a function that updates our search to get the new pages

def ping_acled(api_url): 
    '''
    Takes one parameter search term for API
    '''
    
    response = requests.get(api_url)
    data = response.json()
    return data
    

In [6]:

results = [] # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

while num_results == 500: #if less 500 or 0 we know we have all the results
    print ("starting ", page, " ", num_results) #just to see our progress
    api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'.format(iso,page) #the search
    data = ping_acled(api_url) #call the previous function 
    results.append(data['data']) #store in our results
    count += data['count'] #Track number of results
    num_results = data['count'] #update our condition
    page += 1 #update our page variable
    print ("Total Results ", count) #Track our progress
    



starting  1   500
Total Results  500
starting  2   500
Total Results  1000
starting  3   500
Total Results  1401


In [7]:
#Now I want to put them together into one giant result
super_list = []
for res in results: 
    super_list += res
    print (len(super_list))

500
1000
1401


In [8]:
#convert it into an pandas data frame or just use your data structure and do more stuff
niger_res = pd.DataFrame(super_list)
niger_res.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7228161,562,NIR1452,1452,2020-09-18,2020,1,Violence against civilians,Abduction/forced disappearance,Islamic State (West Africa) and/or Boko Haram ...,...,Waragou,13.5316,12.7309,1,Whatsapp,New media,"On 18 September 2020, presumed ISWAP or Boko H...",0,1600717738,NER
1,7228145,562,NIR1450,1450,2020-09-18,2020,1,Explosions/Remote violence,Remote explosive/landmine/IED,Islamic State (West Africa) and/or Boko Haram ...,...,Bosso,13.6987,13.3095,1,Whatsapp,New media,"On 18 September 2020, a Nigerien army vehicle ...",1,1600717738,NER
2,7228153,562,NIR1453,1453,2020-09-18,2020,1,Violence against civilians,Abduction/forced disappearance,Islamic State (West Africa) and/or Boko Haram ...,...,Youwa,13.4671,12.8197,1,Whatsapp,New media,"On 18 September 2020, presumed ISWAP or Boko H...",0,1600717738,NER
3,7228156,562,NIR1449,1449,2020-09-18,2020,1,Explosions/Remote violence,Remote explosive/landmine/IED,Islamic State (West Africa) and/or Boko Haram ...,...,Baroua Gana,13.8986,13.1544,1,Whatsapp,New media,"On 18 September 2020, a Nigerien army vehicle ...",0,1600717738,NER
4,7228158,562,NIR1451,1451,2020-09-18,2020,1,Violence against civilians,Abduction/forced disappearance,Islamic State (West Africa) and/or Boko Haram ...,...,Makintari,13.5337,12.8181,1,Whatsapp,New media,"On 18 September 2020, presumed ISWAP or Boko H...",0,1600717738,NER


### Do the right thing, take some time to look at the codebook and see what these columns are

In [9]:
niger_res.columns


Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

### Homework --- Make a map of some ACLED Data (absolutely use the code from the Global Terrorism Database exercise)

In [21]:
# PIKE HERE
# Code was taken and manipulated from the GTD Assignment from HW #3

#Imported libraries that we needed

import pandas as pd #pandas places ones data in tables format
from bokeh.plotting import figure, output_notebook, show #builds interactive graphs for python
from bokeh.models import Range1d
import math #this is used in graphic section to use the irrational number pi
output_notebook() #Allows inline plotting for Juptyer notebook

Loading BokehJS ...

In [22]:
# Import Bokeh for plotting on map
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer
tile_provider = get_provider('STAMEN_TERRAIN')

In [25]:
#create pyproj transformer to convert form lat/long to web mercator
transformer = Transformer.from_crs('epsg:4326','epsg:3857')

In [26]:
# Created dictionary to find out how many instances of each event type there were
# My focus is on Attacks against civilians, but this will iterate through and count each different event

attacks_civilians = {} #make an empty datastructure (dictionary) to fill

#This loop goes through each row and counts the number of entries by event type
for index, row in niger_res.iterrows():
    if row["event_type"] in attacks_civilians.keys():
        if int(row["year"]) >= 2014:                  # only intersted in data after 2014
            attacks_civilians[row["event_type"]] += 1 #if event type is in the dictionary add 1 attack
    else:
        attacks_civilians[row["event_type"]] = 1 #add event type to dictionary if not in dictionary

attacks_civilians  #prints how many of each type of event occured fafter 2014

{'Violence against civilians': 445,
 'Explosions/Remote violence': 78,
 'Battles': 269,
 'Strategic developments': 156,
 'Riots': 57,
 'Protests': 90}

In [62]:
# I don't need all the columns anymore, just the latitude, longitude, and event type
# changed latitude/longitude from string to float by replacing the data in niger_map
niger_map = niger_res[["latitude", 'longitude', 'event_type']]
niger_map["latitude"] = niger_map["latitude"].astype(float)
niger_map["longitude"] = niger_map["longitude"].astype(float)
niger_map

# The code will still produce the correct output, despite the caveats; it warns that we are 
# replacing the data without making a copy of it

<ipython-input-62-d5339b86e7eb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  niger_map["latitude"] = niger_map["latitude"].astype(float)
<ipython-input-62-d5339b86e7eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  niger_map["longitude"] = niger_map["longitude"].astype(float)


,latitude,longitude,event_type
0,13.5316,12.7309,Violence against civilians
1,13.6987,13.3095,Explosions/Remote violence
2,13.4671,12.8197,Violence against civilians
3,13.8986,13.1544,Explosions/Remote violence
4,13.5337,12.8181,Violence against civilians
...,...,...,...
1396,21.9400,13.6400,Battles
1397,13.5200,2.1200,Protests
1398,13.3200,12.6100,Riots
1399,13.5200,2.1200,Riots


In [80]:
type(niger_map["latitude"][15])
# quick check that we changed lat./long. successfully

numpy.float64

In [64]:
map_dict = {} # empty dictionary to track event type by lat long

# Iterate through tables and associate group with lat/long
for idx, row in niger_map.iterrows():    
    if row['event_type'] in map_dict.keys(): 
        if math.isnan(row["latitude"]):
            #This counts no data
            if row['event_type'] in nan_count.keys(): 
                nan_count[row['event_type']] += 1 
            else: 
                nan_count[row['event_type']] = 1
        else: 
            #This has to convert the lat/long to a mercator projection 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['event_type']].append([point[0],point[1]]) 
    #BOTH the if an else statement do the same thing but since it is a dictionary one needs to add the event type first
    else:  
        if math.isnan(row["latitude"]):
          nan_count[row['event_type']] = 1
        else: 
            point = transformer.transform(row["latitude"],row["longitude"])
            map_dict[row['event_type']] =[[point[0],point[1]]]
        
map_dict # this shows that each event type has a mercator projection lat/long associated with it

{'Violence against civilians': [[1417197.3053400863, 1520532.3369666068],
  [1427082.476122529, 1513148.2253511536],
  [1426904.3649372598, 1520772.7834778116],
  [744794.1851014761, 1512106.5964148554],
  [1417932.0139793223, 1501589.5025214115],
  [1439104.9811282028, 1539052.9515364086],
  [1446852.8176874144, 1579873.2157059596],
  [744794.1851014761, 1512106.5964148554],
  [390230.4749758205, 1453860.5498781863],
  [1459309.468707182, 1602856.6715477658],
  [1461324.3514905402, 1565303.7343705797],
  [1388009.3348540901, 1477743.4438112022],
  [1412533.0186758484, 1516158.8702557597],
  [1376610.2189968592, 1507871.809251138],
  [1382510.1520089023, 1483185.6373155585],
  [1403738.7789031796, 1496315.30070329],
  [165120.20069366271, 1442600.2815762009],
  [1459309.468707182, 1602856.6715477658],
  [288740.49521959294, 1496921.6111190699],
  [1376610.2189968592, 1507871.809251138],
  [346103.4288253669, 1365961.289581152],
  [1454255.5638251672, 1561760.0898798036],
  [1497247.151

In [76]:
pts = [(12.00,0.00), (20.00,17.00)] #Changed lat and long to represent Niger
bbox = []
for pt in transformer.itransform(pts): 
    bbox.append(pt)

In [77]:
NPA_x = []
NPA_y = []
for pt in map_dict['Violence against civilians']:  # this decides what event type will be plotted (can be changed)
    NPA_x.append(pt[0])
    NPA_y.append(pt[1])

In [78]:
#Plots the bounding box
p = figure(x_range=(bbox[0][0], bbox[1][0]),y_range=(bbox[0][1], bbox[1][1]),x_axis_type="mercator", y_axis_type="mercator")
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
# Places a circle for each converted lat/long attack 
p.circle(x = NPA_x, y = NPA_y, color= "firebrick")

#shows the plot
show(p)

# The final plot shows every instance of Violence against civilians in Niger from 2014 to present from the ACLED data